## Data preparation with PANDAS

In [286]:
import pandas as pd


# Power plant conditions
p_conditions = pd.DataFrame({"Power plant":           ["Ahlen", "Fjället",  "Forsen",   "Kärret"],
                           "Initial reservoir level": [5800e6,  1000e6,     20e6,       13e6    ],
                           "Maximum reservoir level": [7160e6,  1675e6,     27e6,       13e6    ],
                           "Minimum reservoir level": [5800e6,  1000e6,     10e6,       6e6     ],
                           "Discharge capacity":      [540,     135,        975,        680     ],
                           "Power conversion":        [0.52,    1.17,       0.29,       0.05    ],
                           "Maximum spillage":        [820,     930,        360,        400     ],
                           "Local flow":              [177,     28,         8,          29      ],
                          })

# Time conditions
t_conditions = pd.DataFrame({"Time": range(1,10),
                             "Cost": [45, 55, 95, 80, 140, 150, 80, 70, 130]})

# Flow conditions (Connections between the power plants)
f_conditions = pd.DataFrame({"From": ["Ahlen",  "Fjället",  "Forsen"],
                             "To":   ["Forsen", "Forsen",   "Kärret"],
                             "Time": [2,        1,          1]
                            })


## GAMSPy FTW

### Initializing all our Parameters and Varaibales etc

In [287]:

from gamspy import Container, Set, Variable, Parameter, Equation, Sum, Model, Sense, Alias


m = Container()

t = Set(m, name="t", description="time in hours", records=t_conditions['Time']) # time at begining of hour 1, 2, 3, ...
p = Set(m, name="p", description="Power plant", records=p_conditions['Power plant'])

# Create alias for set p
p_up = Alias(m, name="p_up", alias_with=p)

# Parameter and Variables definitions


delay = Parameter(m, name="delay", domain=[p,p], description="Time delay for upstream plants", records=f_conditions[['From', 'To', 'Time']])

prices = Parameter(m, name="prices", domain=t, description="Prices (MWh) at different hours", records=t_conditions[['Time', 'Cost']])

reservoir_init = Parameter(m, name="reservoir_init", domain=p, description="Initial reservoir level", records=p_conditions[['Power plant', 'Initial reservoir level']])
reservoir_max = Parameter(m, name="reservoir_max", domain=p, description="Maximum reservoir level", records=p_conditions[['Power plant', 'Maximum reservoir level']])
reservoir_min = Parameter(m, name="reservoir_min", domain=p, description="Minimum reservoir level", records=p_conditions[['Power plant', 'Minimum reservoir level']])
discharge_max = Parameter(m, name="discharge_max", domain=p, description="Discharge capacity", records=p_conditions[['Power plant', 'Discharge capacity']])
power_conversion = Parameter(m, name="power_conversion", domain=p, description="Power conversion", records=p_conditions[['Power plant', 'Power conversion']])
spillage_max = Parameter(m, name="spillage_max", domain=p, description="Maximum spillage", records=p_conditions[['Power plant', 'Maximum spillage']])
local_flow = Parameter(m, name="local_flow", domain=p, description="Local flow", records=p_conditions[['Power plant', 'Local flow']])

discharge = Variable(m, name="discharge", type="positive", domain=[t,p], description="Discharge rate at each power plant at each time")
spillage = Variable(m, name="spillage", type="positive", domain=[t,p], description="Spillage rate at each power plant at each time")
reservoir_level = Variable(m, name="reservoir_level", domain=[t,p], description="Reservoir level at each power plant at each time")

### Equations and condtions

In [288]:
# Discharge criteria
discharge.up[t,p] = discharge_max[p]

# Spillage criteria
spillage.up[t,p] = spillage_max[p]

# Reservoir level criteria
reservoir_level.lo[t,p] = reservoir_min[p]
reservoir_level.up[t,p] = reservoir_max[p]
reservoir_level.fx[t,p].where[t.first] = reservoir_init[p] # Initial reservoir level should be set to reservoir initial level

# Single reservoir equation for all plants
reservoirs = Equation(m, name="reservoirs", domain=[t,p])
reservoirs[t, p].where[t.ord > 1] = reservoir_level[t,p] == reservoir_level[t.lag(1),p] + 3600 * (
        # Upstream inflows
        Sum(p_up.where[delay[p_up,p]>0], 
            discharge[t.lag(delay[p_up,p]), p_up]
            + spillage[t.lag(delay[p_up,p]), p_up]
        )

        # Local inflow
        + local_flow[p]

        # Outflows
        - discharge[t.lag(1),p]
        - spillage[t.lag(1),p]
    )

"""
#OG CODE RUN IF YOU HAVE A BAD DAY
# Discharge criteria
discharge.up[t,p] = discharge_max[p]

# Spillage criteria
spillage.up[t,p] = spillage_max[p]

# Reservoir level criteria
reservoir_level.lo[t,p] = reservoir_min[p]
reservoir_level.up[t,p] = reservoir_max[p]
reservoir_level.fx[t,p].where[t.first] = reservoir_init[p] # Initial reservoir level should be set to reservoir initial level

# Reservoir Ahlen
res_ahlen = Equation(m, name="res_ahlen", domain=t, description="Reservoir level at Ahlen at different hours")
res_ahlen[t].where[t.ord > 1] = reservoir_level[t-1, 'Ahlen'] + 3600*(local_flow['Ahlen'] - discharge[t-1,'Ahlen'] - spillage[t-1,'Ahlen']) == reservoir_level[t,'Ahlen']

# Reservoir Fjället
res_fjället = Equation(m, name="res_fjallet", domain=t, description="Reservoir level at Fjället at different hours")
res_fjället[t].where[t.ord > 1] = reservoir_level[t-1, 'Fjället'] + 3600*(local_flow['Fjället'] - discharge[t-1,'Fjället'] - spillage[t-1,'Fjället']) == reservoir_level[t,'Fjället']

# Reservoir Forsen
res_forsen = Equation(m, name="res_forsen", domain=t, description="Reservoir level at Forsen at different hours")
res_forsen[t].where[t.ord > 1] = reservoir_level[t-1, 'Forsen'] + 3600*(discharge[t-2,'Ahlen'] + spillage[t-2,'Ahlen'] + discharge[t-1,'Fjället'] + spillage[t-1,'Fjället'] + local_flow['Forsen'] - discharge[t-1,'Forsen'] - spillage[t-1,'Forsen']) == reservoir_level[t,'Forsen']

# Reservoir Kärret
res_kärret = Equation(m, name="res_karret", domain=t, description="Reservoir level at Kärret at different hours")
res_kärret[t].where[t.ord > 1] = reservoir_level[t-1, 'Kärret'] + 3600*(discharge[t-1,'Forsen'] + spillage[t-1,'Forsen'] + local_flow['Kärret'] - discharge[t-1,'Kärret'] - spillage[t-1,'Kärret']) == reservoir_level[t,'Kärret']
"""

'\n#OG CODE RUN IF YOU HAVE A BAD DAY\n# Discharge criteria\ndischarge.up[t,p] = discharge_max[p]\n\n# Spillage criteria\nspillage.up[t,p] = spillage_max[p]\n\n# Reservoir level criteria\nreservoir_level.lo[t,p] = reservoir_min[p]\nreservoir_level.up[t,p] = reservoir_max[p]\nreservoir_level.fx[t,p].where[t.first] = reservoir_init[p] # Initial reservoir level should be set to reservoir initial level\n\n# Reservoir Ahlen\nres_ahlen = Equation(m, name="res_ahlen", domain=t, description="Reservoir level at Ahlen at different hours")\nres_ahlen[t].where[t.ord > 1] = reservoir_level[t-1, \'Ahlen\'] + 3600*(local_flow[\'Ahlen\'] - discharge[t-1,\'Ahlen\'] - spillage[t-1,\'Ahlen\']) == reservoir_level[t,\'Ahlen\']\n\n# Reservoir Fjället\nres_fjället = Equation(m, name="res_fjallet", domain=t, description="Reservoir level at Fjället at different hours")\nres_fjället[t].where[t.ord > 1] = reservoir_level[t-1, \'Fjället\'] + 3600*(local_flow[\'Fjället\'] - discharge[t-1,\'Fjället\'] - spillage[t-

### Obejctive

In [289]:
obj = Sum((t, p), prices[t]*3600*power_conversion[p]*discharge[t,p])

## Solution

In [290]:
flow = Model(m, name="flow", equations=m.getEquations(), objective=obj, problem="LP", sense=Sense.MAX)
flow.solve(solver='CPLEX')

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,1406579020,33,105,LP,CPLEX,0.002


In [291]:
from IPython.display import HTML

def horizontal(dfs):
    html = '<div style="display:flex">'
    for df in dfs:
        html += '<div style="margin-right: 32px">'
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))

horizontal([discharge.pivot(), spillage.pivot(), reservoir_level.pivot()])

,Ahlen,Fjället,Forsen,Kärret
1,0.0,0.0,0.000000,680.0
2,0.0,0.0,0.000000,680.0
3,0.0,0.0,975.000000,680.0
4,0.0,0.0,590.777778,680.0
5,522.0,33.0,975.000000,680.0
6,540.0,135.0,975.000000,680.0
7,177.0,28.0,576.000000,680.0
8,177.0,28.0,213.000000,680.0
9,540.0,135.0,975.000000,680.0
,Ahlen,Fjället,Forsen,Kärret


In [292]:
flow.objective_value

1406579020.0